In [32]:
! pip install pandas plotly

In [2]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df0 = pd.read_csv('data/0_preprocessed.csv')
df0

,timestamp,duration,song,artist,album,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,2019-07-05 03:01:44,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,2019-07-05 03:05:23,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,2019-07-05 03:08:41,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,2019-07-05 03:12:01,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,2024-09-04 13:36:23,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,2024-09-04 14:35:32,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,2024-09-05 13:50:59,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,2024-09-05 16:08:47,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [ ]:
top_100 = df0.groupby(['artist', 'song'])['duration'].sum().reset_index()
top_100 = top_100.sort_values('duration', ascending=False).head(100).reset_index(drop=True)
top_100['duration'] = top_100['duration'].astype(int)
top_100

,artist,song,duration
0,Ludovico Einaudi,Nuvole Bianche,9939
1,Taylor Swift,Enchanted,8423
2,Taylor Swift,Sparks Fly,6485
3,Taylor Swift,Everything Has Changed,3942
4,Taylor Swift,Begin Again,3404
...,...,...,...
95,Taylor Swift,ME!,373
96,Rousseau,Faded,369
97,Wiz Khalifa,See You Again,366
98,Johann Strauss II,"The Blue Danube Waltz, Op. 314",341


In [ ]:
len(top_100['artist'].unique())

37

In [ ]:
top_100.to_csv('2_sankey_top_100.csv', index=False)

Manually add columnn `type` and `genre`

In [3]:
df = pd.read_csv('2_sankey_top_100.csv')
df

,artist,song,duration,type,genre
0,Ludovico Einaudi,Nuvole Bianche,9939,Instrumental,Neo-classical
1,Taylor Swift,Enchanted,8423,Vocal,US-UK
2,Taylor Swift,Sparks Fly,6485,Vocal,US-UK
3,Taylor Swift,Everything Has Changed,3942,Vocal,US-UK
4,Taylor Swift,Begin Again,3404,Vocal,US-UK
...,...,...,...,...,...
95,Taylor Swift,ME!,373,Vocal,US-UK
96,Rousseau,Faded,369,Instrumental,Neo-classical
97,Wiz Khalifa,See You Again,366,Vocal,US-UK
98,Johann Strauss II,"The Blue Danube Waltz, Op. 314",341,Instrumental,Classical


Manually fill in type and genres

In [45]:
import pandas as pd

df2 = pd.read_csv('2_sankey_top_100.csv')
df2

,artist,song,duration,type,genre
0,Ludovico Einaudi,Nuvole Bianche,9939,Instrumental,Neo-classical
1,Taylor Swift,Enchanted,8423,Vocal,US-UK
2,Taylor Swift,Sparks Fly,6485,Vocal,US-UK
3,Taylor Swift,Everything Has Changed,3942,Vocal,US-UK
4,Taylor Swift,Begin Again,3404,Vocal,US-UK
...,...,...,...,...,...
95,Taylor Swift,ME!,373,Vocal,US-UK
96,Rousseau,Faded,369,Instrumental,Neo-classical
97,Wiz Khalifa,See You Again,366,Vocal,US-UK
98,Johann Strauss II,"The Blue Danube Waltz, Op. 314",341,Instrumental,Classical


In [46]:
len(df2['type'].unique()), len(df2['genre'].unique()), len(df2['artist'].unique())

(2, 4, 37)

In [47]:
labels = ['Most-played 100']
labels += df2['type'].unique().tolist() + df2['genre'].unique().tolist() + df2['artist'].unique().tolist()
labels_dict = {label: i for i, label in enumerate(labels)}
len(labels), len(labels_dict)

(44, 44)

In [48]:
links = {}
for i, row in df2.iterrows():
  tmp = ['Most-played 100', row['type'], row['genre'], row['artist']]
  for source, target in zip(tmp, tmp[1:]):
    link = f"{labels_dict[source]}-{labels_dict[target]}"
    if link not in links:
      links[link] = 0
    links[link] += 1

len(links)

43

In [49]:
sources, targets, values = [], [], []
for link, value in links.items():
  source, target = link.split('-')
  sources.append(int(source))
  targets.append(int(target))
  values.append(value)

len(sources), len(targets), len(values)

(43, 43, 43)

In [50]:
import json
data = {
    "labels": labels,
    "sources": sources,
    "targets": targets,
    "values": values
}

data_json = json.dumps(data, indent=4)

with open('2_sankey_top_100.json', 'w') as f:
  f.write(data_json)

In [51]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      hoverinfo = None
      # color = "blue"
    ),
    link = dict(
      source = sources,
      target = targets,
      value = values,
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [52]:
?go.Figure